In [1]:
%matplotlib inline
import seaborn as sns; #sns.set(style="ticks", color_codes=True)
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import scipy
style.use('fast')

sns.set_style('ticks',{'legend.frameon':True} )
sns.set_palette('colorblind')
FIG_SIZE = (12, 3)
font = {'family' : 'Times New Roman',
        'weight' : 'bold',
        'size'   : 22}
plt.rc('font', **font)
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42
plt.rcParams['text.usetex'] = True

PALETTE_BASE = sns.color_palette('muted', n_colors=12)
blue=PALETTE_BASE[0]
green=PALETTE_BASE[1] 
red=PALETTE_BASE[2] 
purple=PALETTE_BASE[3]
version_colors=['orange',red, 'gray']

exclude_prefixes = ('@ryancavanaugh/pkg', 'all-packages-', 'cool-', 'neat-', 'wowdude-', 'npmdoc-', 'npmtest-', 'npm-ghost-',)
exclude_suffixes = ('-cdn',)
exclude_ghost = r'^ghost-\d+$'

# LOAD DATA

In [4]:
# Load data
dep_lag = pd.read_csv('../../jsep2018/csv/packages/technical_lag_clean.csv', 
                    usecols=['package','version','date','kind','dependency','used_version'],
                          sep=';', dtype=object, index_col=None,  error_bad_lines=False)
print(len(dep_lag))
dep_lag.head(2)

51351426


,date,dependency,kind,package,used_version,version
0,2015-11-11,0,runtime,shaybagants-test,0.0.0,1.0.0
1,2015-11-11,0,runtime,shaybagants-test,0.0.0,2.0.0


In [6]:
dep_lag = dep_lag.query('dependency == "lodash"').query('kind == "runtime"')
dep_lag.drop(['dependency','kind'], axis=1, inplace=True)

In [12]:
dep_lag.sort_values(['package','date'], inplace=True)

In [13]:
dep_lag.groupby('package').count[['version']].query('version == 1').reset_index().package

,date,package,used_version,version
31038585,2015-12-04,0.workspace,3.10.1,0.1.0
31038586,2015-12-04,0.workspace,3.10.1,0.1.1
31446606,2015-05-08,00k-node,3.2.0,0.10.13
31184153,2016-09-06,05-npm-test,4.15.0,1.0.0
31343731,2017-10-25,0x-assert,4.17.4,0.0.1


In [15]:
package_out = dep_lag.groupby('package').count()[['version']].query('version == 1').reset_index().package
mask = dep_lag.package.isin(package_out)
dep_lag = dep_lag[~mask]

In [18]:
dep_lag['version_major'] = dep_lag['used_version'].apply(lambda x: x.split('.')[0])

In [24]:
dep_lag['version_major_next'] = dep_lag['version_major'].shift(1)
dep_lag['package_next'] = dep_lag['package'].shift(1)
dep_lag['version_next'] = dep_lag['version'].shift(1)


In [30]:
dep_lag.rename(columns={'package_next':'package_prev',
                        'version_next':'version_prev',
                        'version_major_next':'version_major_prev'},
               inplace=True)

In [32]:
candidates = dep_lag.query('package == package_prev').query('version_major != version_major_prev')

In [41]:
candidates = candidates[['package','version_prev','version','date','version_major_prev','version_major']]

In [42]:
candidates.head()

,package,version_prev,version,date,version_major_prev,version_major
30904692,28,1.9.3,1.9.4,2015-04-23,2,3
30909171,3loc,0.1.0,0.2.0,2016-01-29,3,4
30892479,6to5,2.13.7,3.0.0,2015-01-28,2,3
30892493,6to5-core,2.13.7,3.0.0,2015-01-28,2,3
30996154,6to5ify,4.1.0,4.1.1,2015-02-12,2,3


In [49]:
# Load data
only_npm = pd.read_csv('../../popularity/csv/only_npm.csv', 
                       usecols=['Name','Repository URL'],
                          sep=',', dtype=object, index_col=None,  error_bad_lines=False)
only_npm.rename(columns={'Name':'package',
                        'Repository URL':'repo'},
               inplace=True)
print(len(only_npm))
only_npm.head(2)

699243


,package,repo
0,0,NaN
1,001,NaN


In [50]:
candidates = (candidates.
              merge(only_npm,
                    left_on='package',
                    right_on='package',
                    how='left'
                   )
              .dropna()
             )
len(candidates)

5220

In [56]:
candidates = candidates.query('version_major!="latest"').query('version_major_prev!="latest"')

In [54]:
candidates.to_csv('../../../lodash_techLagDebt_study.csv', sep=',', index=False)

In [57]:
candidates.package.drop_duplicates().size

4112